In [1]:
import numpy as np 
import pandas as pd
import pandas_profiling

In [2]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import StratifiedKFold, RandomizedSearchCV

from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Ridge

In [126]:
train = pd.read_csv("../data/train.csv", delimiter='\t')
test = pd.read_csv("../data/test.csv", delimiter='\t')

In [302]:
report = pandas_profiling.ProfileReport(X_train)

In [303]:
report.to_file("pp_report.html")

In [423]:
##code to add cluster labels as colors
import matplotlib.pyplot as plt

fig,axis = plt.subplots(figsize=(25,25))
plt.scatter(X_train_reduced.T[0], X_train_reduced.T[1], c=cluster_colors)

NameError: name 'cluster_colors' is not defined

In [399]:
len(cat_cols)

235

In [30]:
constant = ["140","164", '0']

In [10]:
test.shape[0] /train.shape[0] 

0.13659016393442622

In [72]:
X.shape

(30500, 346)

In [127]:
X_train = train.drop(constant, axis=1)
y_train = train['0']

In [128]:
X_test = test.drop(constant, axis=1)

# Encoding

In [462]:
cat_cols = []

all_dataframe = pd.concat([train,test])
all_dataframe.drop('0', axis=1, inplace=True)

##code to target encode
cat_cols = [column for column in all_dataframe.columns[1:] if len(all_dataframe[column].value_counts())==2]

In [465]:
import category_encoders as ce

encoder = ce.TargetEncoder(cols=cat_cols)

X_train_encoded = encoder.fit_transform(X_train, y_train)

X_test_encoded = encoder.transform(X_test)

TypeError: fit() missing 1 required positional argument: 'y'

# Validation

In [4]:
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

In [5]:
# SKF
skf = StratifiedKFold(n_splits=8, shuffle=True, random_state=42)

# LightGBM

lgbm_model = lgbm.LGBMClassifier(
    n_estimators=300, 
    colsample_bytree=0.6, 
    subsample=0.7,
    learning_rate=0.045, 
    subsample_for_bin=300000,
    num_leaves=21
)

In [16]:
import lightgbm as lgbm

In [169]:
lgbm_model = lgbm.LGBMClassifier(
    n_estimators=300, 
    colsample_bytree=0.6, 
    subsample=0.7,
    learning_rate=0.045, 
    subsample_for_bin=300000,
    num_leaves=21
)

In [82]:
bad = [str(i) for i in np.arange(196, 210)]
X_train["new_col"] = X_train[bad].sum(axis=1)
X = X_train.drop(bad, axis=1)

In [132]:
cv_iter = skf.split(X, y_train)
lgbm_cv = cross_val_score(lgbm_model, X, y_train, cv=cv_iter, verbose=4, scoring="roc_auc")

[CV]  ................................................................
[CV] ....................... , score=0.7444383283228183, total=   3.2s
[CV]  ................................................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    3.3s remaining:    0.0s


[CV] ....................... , score=0.7413783856261733, total=   3.1s
[CV]  ................................................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    6.4s remaining:    0.0s


[CV] ....................... , score=0.7354391610563012, total=   3.1s
[CV]  ................................................................


[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    9.4s remaining:    0.0s


[CV] ....................... , score=0.7438911706727263, total=   3.2s
[CV]  ................................................................
[CV] ....................... , score=0.7472846772334429, total=   3.3s
[CV]  ................................................................
[CV] ........................ , score=0.746749891286546, total=   3.3s
[CV]  ................................................................
[CV] ....................... , score=0.7406567610355922, total=   3.1s
[CV]  ................................................................
[CV] ....................... , score=0.7462999324132614, total=   3.1s


[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:   25.4s finished


In [133]:
print("CV score: {:.4f}".format(lgbm_cv.mean()))

CV score: 0.7433


In [131]:
print("CV score: {:.4f}".format(lgbm_cv.mean()))

CV score: 0.7433


In [185]:
lgbm_model.fit(X_train, y_train)

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=0.6,
        learning_rate=0.045, max_depth=-1, min_child_samples=20,
        min_child_weight=0.001, min_split_gain=0.0, n_estimators=300,
        n_jobs=-1, num_leaves=21, objective=None, random_state=None,
        reg_alpha=0.0, reg_lambda=0.0, silent=True, subsample=0.7,
        subsample_for_bin=300000, subsample_freq=1)

In [186]:
lgbm_pred = (lgbm_model.predict_proba(X_test)[:, 1])

In [187]:
lgbm_pred

array([0.68186701, 0.46840287, 0.2775374 , ..., 0.29377888, 0.72813411,
       0.3589796 ])

# Dimensionality reduction

In [189]:
from sklearn.decomposition import IncrementalPCA

In [470]:
pca = PCA(n_components=2)

X_train_reduced =pca.fit_transform(X_train)

In [471]:
X_train_reduced

array([[-1.52495000e+04,  8.01546295e-01],
       [-1.52485000e+04,  1.19227282e+00],
       [-1.52475000e+04,  1.30498020e+00],
       ...,
       [ 1.52475000e+04,  1.46025469e+00],
       [ 1.52485000e+04,  1.51746516e+00],
       [ 1.52495000e+04,  1.55954855e+00]])

In [472]:
X_test_reduced = pca.transform(X_test)

In [473]:
cv_iter = skf.split(X_train_reduced, y_train)
lgbm_cv = cross_val_score(lgbm_model, X_train_reduced, y_train, cv=cv_iter, verbose=4, scoring="roc_auc")

[CV]  ................................................................
[CV] ....................... , score=0.6097498035784017, total=   1.2s
[CV]  ................................................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.2s remaining:    0.0s


[CV] ....................... , score=0.6201519150517754, total=   1.1s
[CV]  ................................................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    2.3s remaining:    0.0s


[CV] ........................ , score=0.611042892831435, total=   1.2s
[CV]  ................................................................


[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    3.5s remaining:    0.0s


[CV] ....................... , score=0.6018204871255639, total=   1.2s
[CV]  ................................................................
[CV] ....................... , score=0.6168708510348007, total=   1.2s
[CV]  ................................................................
[CV] ....................... , score=0.5959796258751198, total=   1.1s
[CV]  ................................................................
[CV] ....................... , score=0.6142608991120323, total=   1.2s
[CV]  ................................................................
[CV] ....................... , score=0.6083350850584291, total=   1.1s


[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:    9.4s finished


In [193]:
print("Post DR CV score: {:.4f}".format(lgbm_cv.mean()))

Post DR CV score: 0.7300


# Clusterization

In [478]:
# let's cluster this data and see how it correlates with what we see
from sklearn.cluster import DBSCAN

db = DBSCAN(
    n_jobs=-1,min_samples = 10,eps=2.0,
)

cluster_labels = db.fit_predict(X_train_reduced)

# check statistics
print('Clusters found: {}'.format(np.unique(cluster_labels).shape[0]))
pd.Series(cluster_labels).value_counts()

Clusters found: 1


-1    30500
dtype: int64

# XGBoost

max_depth=5, n_estimators=200, learning_rate=0.065

In [87]:
import xgboost as xgb

In [166]:
xgb_model = xgb.XGBClassifier(
    n_estimators=200,
    learning_rate=0.065,
    max_depth=5,
    random_state=42,
    n_jobs=-1

    #colsample_bytree = 0.7,
    #colsample_bylevel = 0.7,
    
    #max_leaves = 8,
    
    #tree_method = 'hist',
    #grow_policy = 'lossguide'
)

In [95]:
cv_iter = skf.split(X, y_train)
xgb_cv = cross_val_score(xgb_model, X, y_train, cv=cv_iter, verbose=4, scoring="roc_auc")

[CV]  ................................................................
[CV] ....................... , score=0.7411062182138102, total=  43.0s
[CV]  ................................................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   43.0s remaining:    0.0s


[CV] ....................... , score=0.7366868498680329, total=  41.6s
[CV]  ................................................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  1.4min remaining:    0.0s


[CV] ....................... , score=0.7328732127986903, total=  39.5s
[CV]  ................................................................


[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:  2.1min remaining:    0.0s


[CV] ........................ , score=0.737414668341543, total=  41.1s
[CV]  ................................................................
[CV] ....................... , score=0.7443023622343604, total=  40.0s
[CV]  ................................................................
[CV] ....................... , score=0.7410497295811661, total=  40.5s
[CV]  ................................................................
[CV] ....................... , score=0.7402779111203982, total=  39.8s
[CV]  ................................................................
[CV] ....................... , score=0.7442398094374464, total=  38.2s


[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:  5.4min finished


In [498]:
print("CV score: {:.6f}".format(xgb_cv.mean()))

CV score: 0.738548


In [96]:
print("CV score: {:.6f}".format(xgb_cv.mean()))

CV score: 0.739744


In [167]:
xgb_model.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.065, max_delta_step=0,
       max_depth=5, min_child_weight=1, missing=None, n_estimators=200,
       n_jobs=-1, nthread=None, objective='binary:logistic',
       random_state=42, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=True, subsample=1)

In [168]:
xgb_pred = (xgb_model.predict_proba(X_test)[:, 1])

Submission

In [120]:
submission = pd.DataFrame(test.iloc[:, 0])

In [121]:
submission.rename(index=str, columns={'Unnamed: 0': '_ID_'}, inplace=True)

In [122]:
submission['_VAL_'] = xgb_pred

In [123]:
submission

,_ID_,_VAL_
0,0,0.174181
1,1,0.301378
2,2,0.246103
3,3,0.324968
4,4,0.596409
5,5,0.199298
6,6,0.083907
7,7,0.130982
8,8,0.092205
9,9,0.415629


In [124]:
submission.to_csv("xgb2.0.csv", index=False)

In [137]:
lg_predictions = []
xgb_predictions = []

In [141]:
for train, val in skf.split(X_train, y_train):
    lgbm_model.fit(X_train.iloc[train], y_train[train])
    xgb_model.fit(X_train.iloc[train], y_train[train])
    lg_predictions.append([y_train[val], lgbm_model.predict_proba(X_train.iloc[val])[:,1]])
    xgb_predictions.append([y_train[val], xgb_model.predict_proba(X_train.iloc[val])[:,1]])

In [163]:
for i, fold in enumerate(skf.split(X_train, y_train)):
    train, val = fold[0], fold[1]
    X_train.iloc[val, -2] = lg_predictions[i][1]
    X_train.iloc[val, -1] = xgb_predictions[i][1]

In [165]:
X_train.shape

(30500, 344)

In [173]:
from math import sqrt

In [182]:
weighted_pred = lgbm_pred*0.4+xgb_pred*0.6

In [184]:
lgbm_pred

array([0.68186701, 0.46840287, 0.2775374 , ..., 0.29377888, 0.72813411,
       0.3589796 ])

In [183]:
weighted_pred

array([0.74539486, 0.41453112, 0.26942469, ..., 0.31957252, 0.78819929,
       0.34151885])

# Xgbfir

In [257]:
import xgbfir
from os.path import join as pjoin

In [255]:
DATA_DIR = '../data'

In [254]:
train_cols = [col for col in X_train.columns if X_train[col].dtype != 'O']

In [258]:
xgbfir.saveXgbFI(
    xgb_model, 
    feature_names=train_cols, 
    OutputXlsxFile=pjoin(DATA_DIR, "xgbfir_importance.xlsx")
)

In [259]:
candidates = ['338', '339']

In [ ]:
candidates = ['329', '338']

In [493]:
#new fitcha
X_train["338|339_0"] = (X_train["338"]) + (X_train["339"])
X_test["338|339_0"] = (X_test["338"]) + (X_test["339"])

#new fitcha
X_train["338|339_1"] = (X_train["338"]) * (X_train["339"])
X_test["338|339_1"] = (X_test["338"]) * (X_test["339"])

#new fitcha
X_train["338|339_2"] = np.sqrt((X_train["338"]) * (X_train["339"]))
X_test["338|339_2"] = np.sqrt((X_test["338"]) * (X_test["339"]))

X_train["338|339_3"] = ((X_train["338"]) + (X_train["339"])) / 2.0
X_test["338|339_3"] = ((X_test["338"]) + (X_test["339"])) / 2.0



In [499]:
#new fitcha
X_train["338|329_0"] = (X_train["338"]) + (X_train["329"])
X_test["338|329_0"] = (X_test["338"]) + (X_test["329"])

#new fitcha
X_train["338|329_1"] = (X_train["338"]) * (X_train["329"])
X_test["338|329_1"] = (X_test["338"]) * (X_test["329"])

#new fitcha
X_train["338|329_2"] = np.sqrt((X_train["338"]) * (X_train["329"]))
X_test["338|329_2"] = np.sqrt((X_test["338"]) * (X_test["329"]))

X_train["338|329_3"] = ((X_train["338"]) + (X_train["329"])) / 2.0
X_test["338|329_3"] = ((X_test["338"]) + (X_test["329"])) / 2.0

In [510]:
X_train.drop(['338|329_3', '338|329_2', '338|329_1', '338|329_0',
              '338|339_3', '338|339_2', '338|339_1', '338|339_0'], axis=1, inplace=True)

In [514]:
X_test.drop(['338|329_3', '338|329_2', '338|329_1', '338|329_0',
              '338|339_3', '338|339_2', '338|339_1', '338|339_0'], axis=1, inplace=True)

In [521]:
X_test.drop('338|339', axis=1, inplace=True)